# Modeling Food Webs Bioenergetics

## Mass Balance

Conservation of mass: in a closed physical system mass can neither be produced nor destroyed.

**Mass balance** is the application of the physical principle of conservation
of mass to the analysis of systems flux an stocks. In short, by making sure
that inputs and outputs add up, we can study the flux within a system.

$$
m(t+\Delta t) = m(t) + m_{in}(t \rightarrow \Delta t) - m_{out}(t \rightarrow \Delta t)
$$

Inputs $m_{in}(t \rightarrow \Delta t)$ and outputs $m_{out}(t \rightarrow \Delta t)$
can both be decomposed in multiple terms to reflect the diversity of mechanisms
through which mass is transfered between the components of the system.